In [ ]:
from flask import Flask, jsonify, request
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)

# Load the trained model
new_model = load_model("model_weights.h5")

@app.route('/predict', methods=['POST'])
def predict():
    # Check if the POST request contains an image file
    if 'image' not in request.files:
        return jsonify({'error': 'No image file found in request'}), 400

    # Load the image from the POST request
    file = request.files['image']
    img = load_img(file.stream, target_size=(200, 200), grayscale=True)

    # Convert the image to a numpy array
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Normalize the pixel values
    x = x / 255.0

    # Make the prediction
    prediction = new_model.predict(x)

    # Get the class label with the highest probability
    predicted_class = np.argmax(prediction)

    # Return the predicted tumor type as a JSON response
    if predicted_class == 0:
        tumor_type = "Glioma"
    elif predicted_class == 1:
        tumor_type = "Meningioma"
    elif predicted_class == 2:
        tumor_type = "No tumor"
    elif predicted_class == 3:
        tumor_type = "Pituitary"
    else:
        return jsonify({'error': 'Unknown class label'}), 500
    
    response = {'tumor_type': tumor_type}
    return jsonify(response), 200

@app.errorhandler(400)
def bad_request(error):
    return jsonify({'error': 'Bad request'}), 400

@app.errorhandler(500)
def internal_server_error(error):
    return jsonify({'error': 'Internal server error'}), 500

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\Abdo fahmy\anaconda3\lib\site-packages\keras\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(
[2023-04-28 03:37:02,395] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\Abdo fahmy\anaconda3\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Abdo fahmy\anaconda3\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Abdo fahmy\anaconda3\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Abdo fahmy\anaconda3\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Abdo fahmy\AppData\Local\Temp\ipykernel_11164\22